# Nilm RNN Test

In [1]:
from nilm_reader import nilm_reader
import matplotlib.pyplot as plt
%matplotlib inline
from nilm_utils import *
import time
import tensorflow as tf

In [2]:
HIDDEN_SIZE = 200
BATCH_SIZE = 64
NUM_STEPS = 50
SKIP_STEP = 40
TEMPRATURE = 0.7
LR = 0.003
LEN_GENERATED = 3000

In [63]:
def create_rnn(seq, hidden_size=HIDDEN_SIZE):
    cell = tf.contrib.rnn.GRUCell(hidden_size)
    in_state = tf.placeholder_with_default(cell.zero_state(tf.shape(seq)[0], tf.float32), [None, hidden_size])
    # this line to calculate the real length of seq
    # all seq are padded to be of the same length which is NUM_STEPS
    #length = tf.reduce_sum(tf.reduce_max(tf.sign(seq), 2), 1)
    output, out_state = tf.nn.dynamic_rnn(cell, seq, initial_state=in_state)
    #return output, in_state, out_state
    return(0,0,0)

In [64]:
def create_model(seq, temp, hidden=HIDDEN_SIZE):
    output, in_state, out_state = create_rnn(seq, hidden)
    # fully_connected is syntactic sugar for tf.matmul(w, output) + b
    # it will create w and b for us
    #logits = tf.contrib.layers.fully_connected(output, NUM_STEPS, None)
    #loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=logits[:, :-1], labels=seq[:, 1:]))
    # sample the next character from Maxwell-Boltzmann Distribution with temperature temp
    # it works equally well without tf.exp
    #sample = tf.multinomial(tf.exp(logits[:, -1] / temp), 1)[:, 0] 
    #return loss, sample, in_state, out_state
    return(0,0,0,0)

In [65]:
def training(seq, loss, optimizer, global_step, temp, sample, in_state, out_state):
    start = time.time()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        iteration = global_step.eval()
        for batch in reader.read_batch(reader.read_single_meter('fridge', 12, NUM_STEPS, NUM_STEPS), BATCH_SIZE):
            batch_loss, _ = sess.run([loss, optimizer], {seq: batch})
            if (iteration + 1) % SKIP_STEP == 0:
                print('Iter {}. \n    Loss {}. Time {}'.format(iteration, batch_loss, time.time() - start))
                # online_inference(sess, vocab, seq, sample, temp, in_state, out_state)
                start = time.time()
            iteration += 1

In [66]:
def online_inference(sess, seq, sample, temp, in_state, out_state, seed):
    """ Generate sequence one character at a time, based on the previous character
    """
    sentence = seed
    state = None
    for _ in range(LEN_GENERATED):
        batch = sentence[-1]
        feed = {seq: batch, temp: TEMPRATURE}
        # for the first decoder step, the state is None
        if state is not None:
            feed.update({in_state: state})
        index, state = sess.run([sample, out_state], feed)
        sentence += vocab_decode(index, vocab)
    print(sentence)

In [67]:
seq = tf.placeholder(tf.int32, [None, None])
temp = tf.placeholder(tf.float32)
loss, sample, in_state, out_state = create_model(seq, temp)
#global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
#optimizer = tf.train.AdamOptimizer(LR).minimize(loss, global_step=global_step)
#training(seq, loss, optimizer, global_step, temp, sample, in_state, out_state)

ValueError: Shape (?, ?) must have rank at least 3